# Convolutional net finetuning

In [1]:
import numpy as np
import os

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from finetune import load_data, to_categorical

Using TensorFlow backend.


## Just save Inception V3 as model file

In [17]:
model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
#logdir = os.makedirs('inceptionv3/')
#model.save(os.path.join(logdir, 'model.h5'))

In [18]:
model.output.shape.as_list()[-1]

2048

## Settings

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)
base_model.summary()

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# this is the model we will train
model = Model(base_model.input, predictions)
#model.summary()
model.layers.pop()

In [ ]:
model.output_shape

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## Get dataset

In [ ]:
meme_path = 'many_meme_characters/train/'
non_meme_path = '/media/backup/crawling/memes/test2014/'
m_train, m_test, n_train, n_test = load_data(meme_path, non_meme_path, size=2000)

In [ ]:
# Training set
x_train = np.vstack([m_train, n_train])
my_train = np.ones((m_train.shape[0], 1))
ny_train = np.zeros((n_train.shape[0], 1))
y_train = to_categorical(np.vstack([my_train, ny_train]), num_classes=2)

In [ ]:
# Validation set
x_test = np.vstack([m_test, n_test])
my_test = np.ones((m_test.shape[0], 1))
ny_test = np.zeros((n_test.shape[0], 1))
y_test = to_categorical(np.vstack([my_test, ny_test]), num_classes=2)

## Testing while training

In [ ]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

## Callbacks

In [ ]:
logdir = 'inception_log4.0'

In [ ]:
tb = TensorBoard(log_dir=logdir, histogram_freq=1, write_images=True)

## Training

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=32),
                    samples_per_epoch = len(x_train) / 32,
                    nb_epoch=80,
                    callbacks=[tb],
                    #initial_epoch=30,
                    validation_data=test_datagen.flow(x_test, y_test, batch_size=32),
                    nb_val_samples=1600 / 32)
model.save(os.path.join(logdir, 'fine_inception.h5'))

In [ ]:
score = model.evaluate_generator(test_datagen.flow(x_test, y_test, batch_size=32), 32)
print('score:', score)